# EnMAP SNR Experiment Runner

This notebook orchestrates the eight-case SNR experiment (A–H) for EnMAP scenes via the `scripts.snr_experiment` CLI. Each run produces destriping diagnostics, PCA summaries, and SNR curves stored under `notebooks/outputs/enmap/<scene_id>/`.

## Prerequisites

- Install the required Python dependencies (`numpy`, `scipy`, `scikit-learn`, `matplotlib`, `scikit-image`, etc.).
- Provide EnMAP L1B inputs: VNIR & SWIR GeoTIFFs plus `METADATA.XML`. You may supply a directory containing these files or pass explicit paths.
- Optionally configure ROIs, spectral windows, destriping/PCA settings through the scene configuration below.

In [ ]:
import os
import sys
import subprocess
from pathlib import Path
from datetime import datetime
from typing import Iterable, Optional

from IPython.display import Markdown, display

NOTEBOOK_ROOT = Path.cwd().resolve()
REPO_ROOT = NOTEBOOK_ROOT
while not (REPO_ROOT / 'scripts').exists() and REPO_ROOT.parent != REPO_ROOT:
    REPO_ROOT = REPO_ROOT.parent
if not (REPO_ROOT / 'scripts').exists():
    raise RuntimeError('Could not locate repository root containing scripts directory.')
os.environ['PYTHONPATH'] = str(REPO_ROOT)
if str(REPO_ROOT) not in sys.path:
    sys.path.insert(0, str(REPO_ROOT))
PYTHON = sys.executable
DEFAULT_OUTDIR = NOTEBOOK_ROOT / 'outputs'
DEFAULT_OUTDIR.mkdir(parents=True, exist_ok=True)



In [ ]:
display(Markdown(
    f"""\n**Notebook root**: `{NOTEBOOK_ROOT}`  \\n**Python**: `{PYTHON}`  \\n**PYTHONPATH**: `{os.environ['PYTHONPATH']}`\n"""
))


In [ ]:
def run_snr_experiment(
    sensor: str,
    inputs: Iterable[str],
    roi: Optional[str] = None,
    bands: Optional[str] = None,
    cases: Optional[str] = None,
    disable_notch: bool = False,
    extra_args: Optional[Iterable[str]] = None,
    outdir: Optional[str] = None,
):
    """Invoke the consolidated CLI for a single EnMAP scene."""

    target_outdir = Path(outdir) if outdir else DEFAULT_OUTDIR
    target_outdir.mkdir(parents=True, exist_ok=True)

    cmd = [
        PYTHON,
        "-m",
        "scripts.snr_experiment",
        "--sensor",
        sensor,
        "--input",
    ]
    cmd.extend(str(path) for path in inputs)
    if roi:
        cmd.extend(["--roi", roi])
    if bands:
        cmd.extend(["--bands", bands])
    if cases:
        cmd.extend(["--cases", cases])
    if disable_notch:
        cmd.append("--disable-notch")
    cmd.extend(["--outdir", str(target_outdir)])
    if extra_args:
        cmd.extend(str(arg) for arg in extra_args)

    display(Markdown(f"**Running**: `{' '.join(cmd)}`"))
    start = datetime.now()
    result = subprocess.run(cmd, capture_output=True, text=True, env=os.environ.copy())
    duration = datetime.now() - start
    status = "success" if result.returncode == 0 else f"failed (code {result.returncode})"

    display(Markdown(f"- Status: **{status}** in {duration.total_seconds():.1f}s"))
    if result.stdout:
        display(Markdown(f"<details><summary>stdout</summary><pre>{result.stdout}</pre></details>"))
    if result.stderr:
        display(Markdown(f"<details><summary>stderr</summary><pre>{result.stderr}</pre></details>"))

    return result


## Outputs and Interpretation

Running a scene produces `outputs/enmap/<scene_id>/` with several diagnostics:
- **striping_diagnostics.png** – compares plain vs destriped radiance for a representative band.
  - Spatial maps (top) should reveal fewer stripes after processing.
  - Column mean/σ plots (middle) show how equalisation affects column statistics; flatter curves and lower σ indicate success.
  - The FFT panel (bottom) highlights stripe frequencies; a smaller peak means coherent striping was damped.
- **pca_summary_plain.png** and **pca_summary_destriped.png** – PCA breakdown of the cube before/after destriping.
  - Use the explained variance and component spectra to ensure signal structure is preserved.
  - Residual maps (SWIR/VNIR) and spectra reveal whether destriping reduced structured noise.
- **snr_cases_*.csv** – numeric SNR curves for cases A–H. Key columns:
  - `mu`: mean radiance used for SNR, `sigma`: noise estimate (total or random).
  - `snr_median` / `snr_p90`: column-median and 90th percentile SNR for columnwise cases (equal to ROI SNR when `aggregation=roi`).
  - `case`, `sigma_type`, `aggregation` document how the metric was computed.
- **snr_cases_overview.png** – grid of cases A–H for quick comparison between plain/destriped and ROI/columnwise modes.

### How to interpret the results
- Start with the striping diagnostic to verify that destriping reduces column oscillations without creating artefacts.
- Review the PCA summaries to confirm residuals become less structured and that principal components remain physically plausible.
- Compare SNR CSVs or the overview plot: destriping should narrow the gap between ROI and columnwise curves without depressing absolute SNR.
- If you disable equalisation (`--disable-equalize`) or the notch, expect the corresponding panels to look similar – a useful sanity check.


## Configure EnMAP Scenes

Each configuration dictionary is passed to `run_snr_experiment`. Key destriping options (add to `extra_args`):

- `--equalize-scale-strength <0-1>`: blend factor for column gain correction (0 = offsets only, default 0.3).
- `--equalize-scale-cap <>=1>`: clamp gain correction, keep near 1–1.5 to avoid noise amplification.
- `--equalize-poly-order <int>`: polynomial order removed per column (0/1 for gentle detrend, -1 disables).
- `--disable-equalize`: bypass column equalisation (only the FFT notch runs if enabled).
- `--disable-notch`: skip the FFT notch if high-frequency striping is not obvious.

Optimisation tips:
- Inspect `striping_diagnostics.png`: you want column σ to fall and the FFT stripe peak to shrink without new fine-scale noise.
- Begin with `--equalize-scale-strength 0` (offset only). If columns still show gain stripes, raise to 0.1–0.3.
- Reduce `--equalize-poly-order` if you observe over-flattening; use -1 for scenes with genuine illumination gradients.
- Compare SNR columns (cases F/H) against ROI (E/G); destriping should narrow median/P90 gaps while maintaining overall level.

Example entry for the provided test data is commented below—adjust paths and parameters accordingly.

Add destriping parameters inside the `extra_args` list of each scene configuration.


In [ ]:
SCENE_CONFIGS = [
    # Example configuration using the test dataset shipped with the repo:
    {
        "sensor": "enmap",
        "inputs": [
            "/mnt/d/Lavoro/Assegno_Ricerca_Sapienza/CLEAR_UP/CH4_detection/Matched_filter_approach/hygas/test_data/enmap/Agadez_Niger_20220712/L1B-DT0000001584_20220712T104302Z_001_V010502_20251017T093724Z/"
            # or explicitly:
            # "/path/EnMAP/.../SPECTRAL_IMAGE_VNIR.TIF",
            # "/path/EnMAP/.../SPECTRAL_IMAGE_SWIR.TIF",
            # "/path/EnMAP/.../METADATA.XML",
        ],
        # "roi": "x0:x1,y0:y1",  # optional subwindow (columns,rows)
        "bands": "1000:2500",  # optional wavelength range in nm
        "cases": "A,B,C,D,E,F,G,H",
        # add destriping tuning here, e.g.
        "extra_args": [
            "--mask-frac", "0.12",
            "--diff-axis", "columns",
            "--k-pca", "4",
            "--disable-equalize",
            "--equalize-scale-strength", "0.5",
            "--equalize-scale-cap", "1.5",
            "--equalize-poly-order", "1",
        ],
        "disable_notch": True,
    },
]
SCENE_CONFIGS

## Run Experiments

In [ ]:
results = []
for cfg in SCENE_CONFIGS:
    result = run_snr_experiment(**cfg)
    results.append((cfg, result))
    if result.returncode != 0:
        break

results

## Inspect Outputs

The cell below lists the generated CSV/PNG artefacts so you can quickly preview diagnostics and SNR curves.

In [ ]:
from glob import glob
from IPython.display import Image

output_root = (DEFAULT_OUTDIR / "enmap")
output_root.mkdir(parents=True, exist_ok=True)

if output_root.exists():
    csv_paths = sorted(output_root.glob("**/snr_cases_*.csv"))
    overview_pngs = sorted(output_root.glob("**/snr_cases_overview.png"))
    striping_pngs = sorted(output_root.glob("**/striping_diagnostics.png"))
    pca_pngs = sorted(output_root.glob("**/pca_summary_*.png"))

    display(Markdown(
        f"Found {len(csv_paths)} CSV files, {len(overview_pngs)} SNR overview plots, "
        f"{len(striping_pngs)} striping diagnostics, and {len(pca_pngs)} PCA summaries."
    ))

    if overview_pngs:
        display(Markdown("#### SNR overview"))
        for png in overview_pngs:
            display(Markdown(f"**{png.parent.relative_to(output_root)} / {png.name}**"))
            display(Image(filename=str(png)))

    if striping_pngs:
        display(Markdown("#### Striping diagnostics"))
        for png in striping_pngs:
            display(Markdown(f"**{png.parent.relative_to(output_root)} / {png.name}**"))
            display(Image(filename=str(png)))

    if pca_pngs:
        display(Markdown("#### PCA summaries"))
        for png in pca_pngs:
            display(Markdown(f"**{png.parent.relative_to(output_root)} / {png.name}**"))
            display(Image(filename=str(png)))
else:
    display(Markdown("`outputs/enmap` directory not found yet. Run at least one experiment."))
